# 04/05 - Pair Programming Limpieza 1 - Guadalupe & Lara

In [115]:
import pandas as pd
import seaborn as sns
import numpy as np
import re
import matplotlib.pyplot as plt
pd.options.display.max_columns = None
df = pd.read_csv("1.ataques.csv", index_col = 0)
df.head()

,case_number,year,type,country,area,location,activity,name,sex_,age,time,species_,fatal_(y/n),injury,date
0,2018.06.25,2018.0,Boating,usa,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,18h00,White shark,N,"No injury to occupant, outrigger canoe and pad...",25-Jun-2018
1,2018.06.18,2018.0,Unprovoked,usa,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,14h00 -15h00,NaN,N,Minor injury to left thigh,18-Jun-2018
2,2018.06.09,2018.0,Invalid,usa,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,07h45,NaN,N,Injury to left lower leg from surfboard skeg,09-Jun-2018
3,2018.06.08,2018.0,Unprovoked,australia,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,NaN,2 m shark,N,Minor injury to lower leg,08-Jun-2018
4,2018.06.04,2018.0,Provoked,mexico,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,NaN,"Tiger shark, 3m",N,Lacerations to leg & hand shark PROVOKED INCIDENT,04-Jun-2018


# EJERCICIOS

# 1 Columna de species:

Si exploramos esta columna en detalle nos podemos dar cuenta que tenemos muchos valores únicos y esto hace que sea muy difícil trabajar con esta columna. Lo que vamos a hacer es clasificar los tiburones en 5 especies diferentes, las más comunes, que incluyen el tiburón blanco (White), el tiburón tigre (Tiger ), el tiburón gris (Grey), el tiburón limón (Lemon) y el tiburón toro (Bull). 
El resto de las especies las incluiremos en un único grupo que podremos llamar "Unspecified". Ahora nos podemos sentir un poco abrumadas y no saber como enfrentarnos a este reto, pero don't worry, os dejamos por aquí unas 💡 pistas 💡 para que os ayuden a entender cómo hacerlo.
Los valores de las columnas son strings por lo que podremos usar regex para buscar palabras clave en cada celda y asignarlo a una de las categorías que hemos definido previamente.

 *imaginemos que el valor de una celda es el siguiente*

*'White shark, 3.5 m'*

* tendremos que buscar el patrón de regex que nos permita extraer White shark de ese string y que nos devuelva solo White shark. *
* Un patrón que podriamos usar es: *

*patron_blanco = r".*[Ww](hite|HITE).*" # esto podría ser así porque puede estar en mayúsculas o en minúsculas. *

* de la misma forma que hemos sacado el patron para el tiburón blanco, tendremos que sacar los patrones para las otras 4 especies que queremos "encontrar". *

Tendremos que crearnos una función que aplicaremos sobre nuestra columna species_ para que nos devuelva una nueva columna con los valores clasificados en función de los patrones de regex que hayamos definido.

In [116]:
patron_blanco =r".*[Ww](hite|HITE).*"
patron_tigre =r".*[Tt](iger|IGER).*"
patron_gris = r".*[Gg](rey|REY).*"
patron_limon =r".*[Ll](emon|EMON).*"
patron_torito = r".*[Bb](ull|ULL).*"


In [117]:
def funcion (columna):
    
    try:
        if re.findall(patron_blanco,columna) :
            return "white_shark"
        elif re.findall(patron_tigre,columna) :
            return "tiger_shark"
        elif re.findall(patron_gris,columna) :
            return "grey_shark"
        elif re.findall(patron_limon,columna) :
            return "lemon_shark"
        elif re.findall(patron_torito,columna) :
            return "bull_shark"
        else:
            return "unspecified"
    except:
        return "unspecified"

In [118]:
df["species"] = df["species_"].apply(funcion)
df[["species_", "species"]].head(10)

,species_,species
0,White shark,white_shark
1,NaN,unspecified
2,NaN,unspecified
3,2 m shark,unspecified
4,"Tiger shark, 3m",tiger_shark
5,NaN,unspecified
6,Tiger shark,tiger_shark
7,"Lemon shark, 3'",lemon_shark
8,"Bull shark, 6'",bull_shark
9,NaN,unspecified


In [119]:
df["species"].value_counts()

unspecified    5065
white_shark     666
tiger_shark     282
bull_shark      173
grey_shark       65
lemon_shark      37
Name: species, dtype: int64

# EJERCICIO 2: Columna "age"

In [120]:
patron_age = "[0-9]{1,2}" 
df['age'] = df['age'].astype(str)

def limpio(col):
    x = re.findall(patron_age, col)
    try:
        return int(x[0])
    except:
        np.NaN

In [121]:
df["age2"] = df["age"].apply(limpio)

In [122]:
df["age2"].unique()

array([57., 11., 48., nan, 18., 52., 15., 12., 32., 10., 21., 34., 30.,
       60., 33., 29., 54., 41., 37., 56., 19., 25., 69., 38., 55., 35.,
       46., 45., 14., 40., 28., 20., 24., 26., 49., 22.,  7., 31., 17.,
       13., 42.,  3.,  8., 50., 16., 82., 73., 68., 51., 39., 58., 47.,
       61., 65., 36., 66., 43.,  9., 72., 59.,  6., 27., 64., 23., 71.,
       44., 62., 63., 70., 53., 77., 74.,  5., 86., 84., 75., 87., 67.,
        1.,  2., 81., 78.])

In [123]:
df['age2'].dtypes

dtype('float64')

*Una vez que hayáis extraido los números, os daréis cuenta que hay celdas que tienen más de una edad. Tendréis que decidir que hacer en esos casos. Os dejamos por aquí una posible opción usando un método de Pandas que os puede resultar super útil. El **método explode**, aqui.*

-Por último cambiad el tipo de la columna de string a integer.

In [124]:
df["age2"] = df["age2"].astype("Int64", errors = "ignore")
df.dtypes

case_number     object
year           float64
type            object
country         object
area            object
location        object
activity        object
name            object
sex_            object
age             object
time            object
species_        object
fatal_(y/n)     object
injury          object
date            object
species         object
age2             Int64
dtype: object

# 3
Guardad el csv con las columnas limpias para seguir trabajando con este dataframe limpio.

In [125]:
df.to_csv("2.ataques.csv")